In [1]:
import numpy as np
import tensorflow as tf
from datetime import datetime
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# Load and scale the dataset
housing = fetch_california_housing()
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
m, n = scaled_housing_data.shape
scaled_housing_data_plus_bias = np.c_[np.ones((scaled_housing_data.shape[0], 1)), scaled_housing_data]

# Hyperparameters
n_epochs = 1000
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

# Log directory setup for TensorBoard
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

# Convert data to TensorFlow constants and variables with the same dtype (float32)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0, dtype=tf.float32), name="theta")

# Prepare logging for TensorBoard
file_writer = tf.summary.create_file_writer(logdir)

# Function to fetch batches
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch.astype(np.float32), y_batch.astype(np.float32)  # Ensure batch data is float32

# Gradient Descent Optimization with Mini-Batch Gradient Descent
for epoch in range(n_epochs):
    for batch_index in range(n_batches):
        X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
        
        # Perform gradient descent with GradientTape
        with tf.GradientTape() as tape:
            y_pred = tf.matmul(X_batch, theta)  # Both X_batch and theta are float32 now
            error = y_pred - y_batch
            mse = tf.reduce_mean(tf.square(error))

        gradients = tape.gradient(mse, [theta])
        theta.assign_sub(learning_rate * gradients[0])

        # Log MSE every 10 batches
        if batch_index % 10 == 0:
            step = epoch * n_batches + batch_index
            with file_writer.as_default():
                tf.summary.scalar('MSE', mse, step=step)

# Close the file writer when training is done
file_writer.close()

# Final theta value
print("Best theta:", theta.numpy())


Best theta: [[ 2.0714476 ]
 [ 0.8462012 ]
 [ 0.11558536]
 [-0.26835835]
 [ 0.32982785]
 [ 0.00608357]
 [ 0.07052909]
 [-0.8798858 ]
 [-0.86342514]]
